## Question2

**Step 1: run the docker compose**

**Step 2: open the center control**

**Step 3: open a new termial**

- run `python producer.py` 

Refer to producer.py

## Producer

In [ ]:
import socket
from confluent_kafka import Producer, Consumer, KafkaError
import os
from googleapiclient.discovery import build

# Kafka settings
BROKER = 'localhost:9092'
REQUEST_TOPIC = 'youtube_requests'
RESPONSE_TOPIC = 'youtube_topic'

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cmu-key.json"
youtube = build('youtube', 'v3')

# Function to create a Kafka producer
def create_kafka_producer(broker):
    conf = {
        'bootstrap.servers': broker,
        'client.id': socket.gethostname()
    }
    return Producer(conf)

# Function to create a Kafka consumer
def create_kafka_consumer(broker, group_id, topic):
    conf = {
        'bootstrap.servers': broker,
        'group.id': group_id,
        'auto.offset.reset': 'earliest',
        'client.id': socket.gethostname()
    }
    consumer = Consumer(conf)
    consumer.subscribe([topic])
    return consumer

# Function to get video data
def get_most_popular_videos(video_list):
    request = youtube.videos().list(part="snippet,statistics", id=video_list, maxResults=5)
    response = request.execute()
    videos = []

    for item in response.get('items', []):
        video_title = item['snippet']['title']
        video_like = int(item['statistics'].get('likeCount', 0))
        video_id = item['id']
        video_data = {
            'id': video_id,
            'title': video_title,
            'likeCount': video_like,
        }
        videos.append(video_data)

    return videos

# Main function for producer to process requests
def process_requests():
    producer = create_kafka_producer(BROKER)
    consumer = create_kafka_consumer(BROKER, 'producer_group', REQUEST_TOPIC)

    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        video_list = msg.value().decode('utf-8')
        print(f"Received video list: {video_list}")

        videos = get_most_popular_videos(video_list)
        if not videos:
            continue

        max_like = max(videos, key=lambda item: item['likeCount'])
        result = f"{max_like['id']}; {max_like['title']}; {max_like['likeCount']} likes"

        # Send the response back to the consumer
        producer.produce(RESPONSE_TOPIC, key="The most popular video", value=result)
        producer.flush()

if __name__ == "__main__":
    process_requests()


**Step 4: open a new terminal**

- run `streamlit run consumer.py`

Refer to consumer.py

## Consumer

In [ ]:
import streamlit as st
from confluent_kafka import Consumer, Producer, KafkaError
import socket
import datetime

# Kafka settings
BROKER = 'localhost:9092'
REQUEST_TOPIC = 'youtube_requests'
RESPONSE_TOPIC = 'youtube_topic'
GROUP_ID = 'analytics'

# Function to create a Kafka consumer
def create_kafka_consumer(broker, group_id, topic):
    conf = {
        'bootstrap.servers': broker,
        'group.id': group_id,
        'auto.offset.reset': 'earliest',
        'client.id': socket.gethostname()
    }
    consumer = Consumer(conf)
    consumer.subscribe([topic])
    return consumer

# Function to create a Kafka producer
def create_kafka_producer(broker):
    conf = {
        'bootstrap.servers': broker,
        'client.id': socket.gethostname()
    }
    return Producer(conf)

# Streamlit app
def display_kafka_data():
    consumer = create_kafka_consumer(BROKER, GROUP_ID, RESPONSE_TOPIC)
    producer = create_kafka_producer(BROKER)

    st.write("Listening to Kafka topic:", RESPONSE_TOPIC)
    input_ids = st.text_input('Enter up to five YouTube video IDs (comma-separated)')
    button = st.button("Submit")

    if button and input_ids:
        # Send request to producer
        producer.produce(REQUEST_TOPIC, value=input_ids)
        producer.flush()
        st.write("Request sent. Waiting for response...")

        # Poll for the response
        while True:
            msg = consumer.poll(timeout=1.0)
            if msg is None:
                continue
            if msg.error():
                st.error(msg.error())
                break

            key = msg.key().decode('utf-8')
            value = msg.value().decode('utf-8').split(';')

            # Display the result
            st.markdown(f"🚀 **{key}** ")
            st.markdown(f"**ID:** {value[0]} **Title:** {value[1]}")
            st.markdown(f"**Likes:** {value[2]}")
            break

    consumer.close()

if __name__ == "__main__":
    st.title("YouTube Video Popularity Checker")
    display_kafka_data()

**Result**

Refer to q2_1.png, q2_2.png, q2_3.png, q2_4.png

*Reference*

- https://docs.streamlit.io/develop/api-reference/widgets/st.text_input
- https://developers.google.com/youtube/v3/docs/videos/list
- https://docs.confluent.io/platform/current/clients/confluent-kafka-python/html/index.html#pythonclient-producer